In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
 
#폰트 설정          
from matplotlib import rc  
%matplotlib inline          
rc('font', family='AppleGothic' )
plt.rcParams['axes.unicode_minus'] = False  


In [2]:
#전처리 함수
def prepro():   
    #데이터 불러오기
    dir='/Users/yuchanghee/Desktop/DACON/Parking_Demand/data/'
    train= pd.read_csv(dir+'train.csv') 
    test= pd.read_csv(dir+'test.csv') 
    age_gender_info=pd.read_csv(dir+'age_gender_info.csv')
    sample_submission=pd.read_csv(dir+'sample_submission.csv')
    train=train.rename({'도보 10분거리 내 지하철역 수(환승노선 수 반영)': '지하철', '도보 10분거리 내 버스정류장 수': '버스'},axis=1)
    test=test.rename({'도보 10분거리 내 지하철역 수(환승노선 수 반영)': '지하철', '도보 10분거리 내 버스정류장 수': '버스'},axis=1)
    age_gender_info=age_gender_info.set_index('지역')
    age_gender_info.loc['전체 분포 평균']=age_gender_info.mean()        
    age_gender_info.loc['광역시 평균']=age_gender_info[age_gender_info.index.str.contains('시')].mean()
    age_gender_info.loc['도 평균']=age_gender_info[age_gender_info.index.str.contains('도')].mean()

    #데이터 concat, 중복제거 
    total=pd.concat([train,test],axis=0) 
    total.reset_index(drop=True, inplace=True) 
    total=total.drop_duplicates().reset_index(drop=True) 

    #null 처리
    #1. 자격유형 null
    total.loc[(total['자격유형'].isnull()) & (total['단지코드']=='C2411'), '자격유형']='A'
    total.loc[(total['자격유형'].isnull()) & (total['단지코드']=='C2253'), '자격유형']='C'

    #2. 임대료, 임대보증금 null 
    total['임대료'].replace('-',np.nan,inplace=True)  
    total['임대보증금'].replace('-',np.nan,inplace=True)
    total['임대료']=total['임대료'].astype(np.float64)   
    total['임대보증금']=total['임대보증금'].astype(np.float64) 

    #2.1 아파트 - 지역과 공급유형에 따라 임대료, 임대보증금 처리
    total.loc[total['공급유형']=='장기전세', '임대료']=0 #장기전세는 장기전세는 임대료가 없으므로 0
    total.loc[2707,'임대보증금']=5787000.0 
    total.loc[2707,'임대료']=79980.0 
    total.loc[2709,'임대보증금']=11574000.0 
    total.loc[2709,'임대료']=159960.0
    total.loc[(total['지역']=='대전광역시') & (total['공급유형']=='공공분양'), ['임대료','임대보증금']]=0  #공공분양의 경우 보증금, 임대료 없으므로 총분양가 변수 생성 
    total['총분양가']=0.0
    total.loc[(total['지역']=='대전광역시') & (total['공급유형']=='공공분양'), '총분양가']=[221640000, 217200000, 221640000, 251220000, 251220000, 251260000, 251320000]
    total.loc[(total['임대건물구분']=='아파트') & (total['임대보증금'].isnull()) & (total['지역'] == '부산광역시'), '임대보증금'] = [7000000,7000000,7600000,14800000,23100000]  
    total.loc[(total['임대건물구분']=='아파트') & (total['임대료'].isnull()) & (total['지역'] == '부산광역시'), '임대료']= [135000, 135000, 142000, 198000, 259000]    
    total.loc[(total['임대건물구분']=='아파트') & (total['임대보증금'].isnull()) & (total['지역'] == '대구광역시'), '임대보증금'] = [10847000, 10847000, 17338000] 
    total.loc[(total['임대건물구분']=='아파트') & (total['임대료'].isnull()) & (total['지역'] == '대구광역시'), '임대료'] = [138600, 138600, 197500]  
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='행복주택') & (total['임대보증금'].isnull()) & (total['전용면적']==16.94) , '임대보증금']=14965000.0 
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='행복주택') & (total['임대료'].isnull()) & (total['전용면적']==16.94) , '임대료']=70915.0 
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='행복주택') & (total['임대보증금'].isnull()) & (total['전용면적']==36.770) , '임대보증금']=34140000.000
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='행복주택') & (total['임대료'].isnull()) & (total['전용면적']==36.770) , '임대료']=167000.000  
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='행복주택') & (total['임대보증금'].isnull()) & (total['전용면적']==26.850) , '임대보증금'] = 25000000.000 
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='행복주택') & (total['임대료'].isnull()) & (total['전용면적']==26.850) , '임대료'] = 130000.000 
    total.loc[(total['지역']=='강원도') & (total['공급유형']=='행복주택') & total['임대보증금'].isnull() , '임대보증금']= [13181000, 19700000, 19150000, 21679000]  
    total.loc[(total['지역']=='강원도') & (total['공급유형']=='행복주택') & total['임대료'].isnull() , '임대료'] = [65500, 96000, 94000, 105000] 
    total.loc[(total['지역']=='강원도') & (total['공급유형']=='영구임대') & total['임대보증금'].isnull(), '임대보증금']=[2129000, 2902000]
    total.loc[(total['지역']=='강원도') & (total['공급유형']=='영구임대') & total['임대료'].isnull(), '임대료']=[42350, 57730]

    #2.2 상가 - 

    #3. 버스 - 임대보증금, 등록차량수와 상관성이 있으므로 그에 따라 대체 
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='공공임대(10년)') & (total['지하철'].isnull()) , '지하철']=0
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='공공임대(10년)') &(total['버스'].isnull()), '버스'] = 3

    #4. 지하철 - 일괄적으로 0처리 
    total['지하철']=total['지하철'].fillna(0)  
    total['지하철'].value_counts(dropna=False)
    
    #성별, 세대 merge
    total_age=total.merge(age_gender_info.reset_index(), on='지역', how='inner') 
    return total, total_age

In [3]:
total, total_age=prepro()

In [4]:
total_age.isnull().sum()

단지코드           0
총세대수           0
임대건물구분         0
지역             0
공급유형           0
전용면적           0
전용면적별세대수       0
공가수            0
자격유형           0
임대보증금        397
임대료          397
지하철            0
버스             0
단지내주차면수        0
등록차량수        949
총분양가           0
10대미만(여자)      0
10대미만(남자)      0
10대(여자)        0
10대(남자)        0
20대(여자)        0
20대(남자)        0
30대(여자)        0
30대(남자)        0
40대(여자)        0
40대(남자)        0
50대(여자)        0
50대(남자)        0
60대(여자)        0
60대(남자)        0
70대(여자)        0
70대(남자)        0
80대(여자)        0
80대(남자)        0
90대(여자)        0
90대(남자)        0
100대(여자)       0
100대(남자)       0
dtype: int64

In [10]:
#상가 null 처리
#해당 지역
total[total['임대보증금'].isnull()]['지역'].value_counts()

부산광역시      152
대전광역시       71
충청남도        59
경상남도        45
강원도         39
울산광역시       28
제주특별자치도      3
Name: 지역, dtype: int64

In [20]:
#1. 부산광역시 
total[(total['지역']=='부산광역시') & (total['임대보증금'].isnull())]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수,총분양가
271,C2135,1116,상가,부산광역시,임대상가,14.10,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
272,C2135,1116,상가,부산광역시,임대상가,18.98,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
273,C2135,1116,상가,부산광역시,임대상가,19.36,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
274,C2135,1116,상가,부산광역시,임대상가,21.46,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
275,C2135,1116,상가,부산광역시,임대상가,22.83,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2761,C2676,2572,상가,부산광역시,임대상가,54.60,1,25.0,D,NaN,NaN,0.0,2.0,418.0,NaN,0.0
2762,C2676,2572,상가,부산광역시,임대상가,54.61,1,25.0,D,NaN,NaN,0.0,2.0,418.0,NaN,0.0
2763,C2676,2572,상가,부산광역시,임대상가,75.98,1,25.0,D,NaN,NaN,0.0,2.0,418.0,NaN,0.0
2764,C2676,2572,상가,부산광역시,임대상가,404.65,1,25.0,D,NaN,NaN,0.0,2.0,418.0,NaN,0.0


In [21]:
total[total['단지코드']=='C2135']  #부산금곡1

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수,총분양가
268,C2135,1116,아파트,부산광역시,영구임대,26.37,358,10.0,C,9363000.0,124390.0,1.0,3.0,262.0,127.0,0.0
269,C2135,1116,아파트,부산광역시,영구임대,26.37,294,10.0,C,9363000.0,124390.0,1.0,3.0,262.0,127.0,0.0
270,C2135,1116,아파트,부산광역시,영구임대,31.32,447,10.0,C,10692000.0,137660.0,1.0,3.0,262.0,127.0,0.0
271,C2135,1116,상가,부산광역시,임대상가,14.10,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
272,C2135,1116,상가,부산광역시,임대상가,18.98,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
273,C2135,1116,상가,부산광역시,임대상가,19.36,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
274,C2135,1116,상가,부산광역시,임대상가,21.46,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
275,C2135,1116,상가,부산광역시,임대상가,22.83,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
276,C2135,1116,상가,부산광역시,임대상가,23.40,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
277,C2135,1116,상가,부산광역시,임대상가,27.55,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0


In [24]:
total.loc[(total['단지코드']=='C2135') & (total['전용면적']==23.4), '임대료']=265000 
total.loc[(total['단지코드']=='C2135') & (total['전용면적']==23.4), '임대보증금']=6360000  
 
total.loc[(total['단지코드']=='C2135') & (total['전용면적']==18.98), '임대료']=133000
total.loc[(total['단지코드']=='C2135') & (total['전용면적']==18.98), '임대보증금']=3192000


 

276   NaN
Name: 임대료, dtype: float64

In [26]:
total[(total['단지코드']=='C2135') & (total['공급유형']=='임대상가')]['전용면적'].value_counts()

14.10     1
18.98     1
19.36     1
21.46     1
22.83     1
23.40     1
27.55     1
29.17     1
31.92     1
32.60     1
36.57     1
50.08     1
240.22    1
Name: 전용면적, dtype: int64